# Import and Export Data for Santa Catarina

We want to answer the folowing questions:

* What are the top 3 most exported products by State?
* What are the top 3 most imported products by State?
* What are the top 3 most exported products in each month of 2019 by State?
* What is the percentage of total national exports by State in 2019?
* What is the percentage of total national imports by State in 2019?
* Prediction of value of top 3 exported products by month from Santa Catarina to each target Country.
* Prediction of value of top 3 imported products by month from Santa Catarina from each source Country.

## Contents





## Part 1: Data Wrangling

First step to address our analysis will be to download the files, unzip them, clean them and finally organize the best way possible. I have decided to store the link to the government website and download it as we execute the notebook instead of uploading the file as Git LFS because those file locations **should not** change at all. Considering we are betting on the Brazilian government efficiency, we will store the MD5 of the used files "just in case" they ever change for any reason. One faster way to do this is simply download the zip files yourself and put them in the data folder.

In [16]:
#imports
import pandas as pd
import requests
import numpy as np
import seaborn as sns
import os
import asyncio
import hashlib
import base64
import zipfile

EXP_MD5 = b'qQlAf4t9CVrpQ9h+fMMqfw=='
IMP_MD5 = b'X5m1GyzT+AlRGNciPVHrOA=='

In [7]:
#download files (have patience young padawan...)
async def download_to_data_folder(file_url:str)->str:
    local_filename = 'data/'+file_url.split('/')[-1]
    with requests.get(file_url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            downloaded = 0
            for chunk in r.iter_content(chunk_size=32768): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    downloaded += 32768
                    print(file_url.split('/')[-1]+'  '+str(downloaded),end='\r')
    return local_filename

async def download_all_files()->list:
    request_list = []
    request_list.append(download_to_data_folder('http://www.mdic.gov.br/balanca/bd/comexstat-bd/ncm/EXP_COMPLETA.zip'))
    request_list.append(download_to_data_folder('http://www.mdic.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip'))
    return asyncio.gather(*request_list)

if not os.path.exists('data/EXP_COMPLETA.zip'):
    file_list = await download_all_files()

EXP_COMPLETA.zip  32768
EXP_COMPLETA.zip  65536
EXP_COMPLETA.zip  98304
EXP_COMPLETA.zip  131072
EXP_COMPLETA.zip  163840
EXP_COMPLETA.zip  196608
EXP_COMPLETA.zip  229376
EXP_COMPLETA.zip  262144
EXP_COMPLETA.zip  294912
EXP_COMPLETA.zip  327680
EXP_COMPLETA.zip  360448
EXP_COMPLETA.zip  393216
EXP_COMPLETA.zip  425984
EXP_COMPLETA.zip  458752
EXP_COMPLETA.zip  491520
EXP_COMPLETA.zip  524288
EXP_COMPLETA.zip  557056
EXP_COMPLETA.zip  589824
EXP_COMPLETA.zip  622592
EXP_COMPLETA.zip  655360
EXP_COMPLETA.zip  688128
EXP_COMPLETA.zip  720896
EXP_COMPLETA.zip  753664
EXP_COMPLETA.zip  786432
EXP_COMPLETA.zip  819200
EXP_COMPLETA.zip  851968
EXP_COMPLETA.zip  884736
EXP_COMPLETA.zip  917504
EXP_COMPLETA.zip  950272
EXP_COMPLETA.zip  983040
EXP_COMPLETA.zip  1015808
EXP_COMPLETA.zip  1048576
EXP_COMPLETA.zip  1081344
EXP_COMPLETA.zip  1114112
EXP_COMPLETA.zip  1146880
EXP_COMPLETA.zip  1179648
EXP_COMPLETA.zip  1212416
EXP_COMPLETA.zip  1245184
EXP_COMPLETA.zip  1277952
EXP_COMPLETA.zip  1

EXP_COMPLETA.zip  10846208
EXP_COMPLETA.zip  10878976
EXP_COMPLETA.zip  10911744
EXP_COMPLETA.zip  10944512
EXP_COMPLETA.zip  10977280
EXP_COMPLETA.zip  11010048
EXP_COMPLETA.zip  11042816
EXP_COMPLETA.zip  11075584
EXP_COMPLETA.zip  11108352
EXP_COMPLETA.zip  11141120
EXP_COMPLETA.zip  11173888
EXP_COMPLETA.zip  11206656
EXP_COMPLETA.zip  11239424
EXP_COMPLETA.zip  11272192
EXP_COMPLETA.zip  11304960
EXP_COMPLETA.zip  11337728
EXP_COMPLETA.zip  11370496
EXP_COMPLETA.zip  11403264
EXP_COMPLETA.zip  11436032
EXP_COMPLETA.zip  11468800
EXP_COMPLETA.zip  11501568
EXP_COMPLETA.zip  11534336
EXP_COMPLETA.zip  11567104
EXP_COMPLETA.zip  11599872
EXP_COMPLETA.zip  11632640
EXP_COMPLETA.zip  11665408
EXP_COMPLETA.zip  11698176
EXP_COMPLETA.zip  11730944
EXP_COMPLETA.zip  11763712
EXP_COMPLETA.zip  11796480
EXP_COMPLETA.zip  11829248
EXP_COMPLETA.zip  11862016
EXP_COMPLETA.zip  11894784
EXP_COMPLETA.zip  11927552
EXP_COMPLETA.zip  11960320
EXP_COMPLETA.zip  11993088
EXP_COMPLETA.zip  12025856
E

EXP_COMPLETA.zip  20807680
EXP_COMPLETA.zip  20840448
EXP_COMPLETA.zip  20873216
EXP_COMPLETA.zip  20905984
EXP_COMPLETA.zip  20938752
EXP_COMPLETA.zip  20971520
EXP_COMPLETA.zip  21004288
EXP_COMPLETA.zip  21037056
EXP_COMPLETA.zip  21069824
EXP_COMPLETA.zip  21102592
EXP_COMPLETA.zip  21135360
EXP_COMPLETA.zip  21168128
EXP_COMPLETA.zip  21200896
EXP_COMPLETA.zip  21233664
EXP_COMPLETA.zip  21266432
EXP_COMPLETA.zip  21299200
EXP_COMPLETA.zip  21331968
EXP_COMPLETA.zip  21364736
EXP_COMPLETA.zip  21397504
EXP_COMPLETA.zip  21430272
EXP_COMPLETA.zip  21463040
EXP_COMPLETA.zip  21495808
EXP_COMPLETA.zip  21528576
EXP_COMPLETA.zip  21561344
EXP_COMPLETA.zip  21594112
EXP_COMPLETA.zip  21626880
EXP_COMPLETA.zip  21659648
EXP_COMPLETA.zip  21692416
EXP_COMPLETA.zip  21725184
EXP_COMPLETA.zip  21757952
EXP_COMPLETA.zip  21790720
EXP_COMPLETA.zip  21823488
EXP_COMPLETA.zip  21856256
EXP_COMPLETA.zip  21889024
EXP_COMPLETA.zip  21921792
EXP_COMPLETA.zip  21954560
EXP_COMPLETA.zip  21987328
E

EXP_COMPLETA.zip  30801920
EXP_COMPLETA.zip  30834688
EXP_COMPLETA.zip  30867456
EXP_COMPLETA.zip  30900224
EXP_COMPLETA.zip  30932992
EXP_COMPLETA.zip  30965760
EXP_COMPLETA.zip  30998528
EXP_COMPLETA.zip  31031296
EXP_COMPLETA.zip  31064064
EXP_COMPLETA.zip  31096832
EXP_COMPLETA.zip  31129600
EXP_COMPLETA.zip  31162368
EXP_COMPLETA.zip  31195136
EXP_COMPLETA.zip  31227904
EXP_COMPLETA.zip  31260672
EXP_COMPLETA.zip  31293440
EXP_COMPLETA.zip  31326208
EXP_COMPLETA.zip  31358976
EXP_COMPLETA.zip  31391744
EXP_COMPLETA.zip  31424512
EXP_COMPLETA.zip  31457280
EXP_COMPLETA.zip  31490048
EXP_COMPLETA.zip  31522816
EXP_COMPLETA.zip  31555584
EXP_COMPLETA.zip  31588352
EXP_COMPLETA.zip  31621120
EXP_COMPLETA.zip  31653888
EXP_COMPLETA.zip  31686656
EXP_COMPLETA.zip  31719424
EXP_COMPLETA.zip  31752192
EXP_COMPLETA.zip  31784960
EXP_COMPLETA.zip  31817728
EXP_COMPLETA.zip  31850496
EXP_COMPLETA.zip  31883264
EXP_COMPLETA.zip  31916032
EXP_COMPLETA.zip  31948800
EXP_COMPLETA.zip  31981568
E

EXP_COMPLETA.zip  41287680
EXP_COMPLETA.zip  41320448
EXP_COMPLETA.zip  41353216
EXP_COMPLETA.zip  41385984
EXP_COMPLETA.zip  41418752
EXP_COMPLETA.zip  41451520
EXP_COMPLETA.zip  41484288
EXP_COMPLETA.zip  41517056
EXP_COMPLETA.zip  41549824
EXP_COMPLETA.zip  41582592
EXP_COMPLETA.zip  41615360
EXP_COMPLETA.zip  41648128
EXP_COMPLETA.zip  41680896
EXP_COMPLETA.zip  41713664
EXP_COMPLETA.zip  41746432
EXP_COMPLETA.zip  41779200
EXP_COMPLETA.zip  41811968
EXP_COMPLETA.zip  41844736
EXP_COMPLETA.zip  41877504
EXP_COMPLETA.zip  41910272
EXP_COMPLETA.zip  41943040
EXP_COMPLETA.zip  41975808
EXP_COMPLETA.zip  42008576
EXP_COMPLETA.zip  42041344
EXP_COMPLETA.zip  42074112
EXP_COMPLETA.zip  42106880
EXP_COMPLETA.zip  42139648
EXP_COMPLETA.zip  42172416
EXP_COMPLETA.zip  42205184
EXP_COMPLETA.zip  42237952
EXP_COMPLETA.zip  42270720
EXP_COMPLETA.zip  42303488
EXP_COMPLETA.zip  42336256
EXP_COMPLETA.zip  42369024
EXP_COMPLETA.zip  42401792
EXP_COMPLETA.zip  42434560
EXP_COMPLETA.zip  42467328
E

...

IMP_COMPLETA.zip  414547968
IMP_COMPLETA.zip  414580736
IMP_COMPLETA.zip  414613504
IMP_COMPLETA.zip  414646272
IMP_COMPLETA.zip  414679040
IMP_COMPLETA.zip  414711808
IMP_COMPLETA.zip  414744576
IMP_COMPLETA.zip  414777344
IMP_COMPLETA.zip  414810112
IMP_COMPLETA.zip  414842880
IMP_COMPLETA.zip  414875648
IMP_COMPLETA.zip  414908416
IMP_COMPLETA.zip  414941184
IMP_COMPLETA.zip  414973952
IMP_COMPLETA.zip  415006720
IMP_COMPLETA.zip  415039488
IMP_COMPLETA.zip  415072256
IMP_COMPLETA.zip  415105024
IMP_COMPLETA.zip  415137792
IMP_COMPLETA.zip  415170560
IMP_COMPLETA.zip  415203328
IMP_COMPLETA.zip  415236096
IMP_COMPLETA.zip  415268864
IMP_COMPLETA.zip  415301632
IMP_COMPLETA.zip  415334400
IMP_COMPLETA.zip  415367168
IMP_COMPLETA.zip  415399936
IMP_COMPLETA.zip  415432704
IMP_COMPLETA.zip  415465472
IMP_COMPLETA.zip  415498240
IMP_COMPLETA.zip  415531008
IMP_COMPLETA.zip  415563776
IMP_COMPLETA.zip  415596544
IMP_COMPLETA.zip  415629312
IMP_COMPLETA.zip  415662080
IMP_COMPLETA.zip  41

IMP_COMPLETA.zip  424345600
IMP_COMPLETA.zip  424378368
IMP_COMPLETA.zip  424411136
IMP_COMPLETA.zip  424443904
IMP_COMPLETA.zip  424476672
IMP_COMPLETA.zip  424509440
IMP_COMPLETA.zip  424542208
IMP_COMPLETA.zip  424574976
IMP_COMPLETA.zip  424607744
IMP_COMPLETA.zip  424640512
IMP_COMPLETA.zip  424673280
IMP_COMPLETA.zip  424706048
IMP_COMPLETA.zip  424738816
IMP_COMPLETA.zip  424771584
IMP_COMPLETA.zip  424804352
IMP_COMPLETA.zip  424837120
IMP_COMPLETA.zip  424869888
IMP_COMPLETA.zip  424902656
IMP_COMPLETA.zip  424935424
IMP_COMPLETA.zip  424968192
IMP_COMPLETA.zip  425000960
IMP_COMPLETA.zip  425033728
IMP_COMPLETA.zip  425066496
IMP_COMPLETA.zip  425099264
IMP_COMPLETA.zip  425132032
IMP_COMPLETA.zip  425164800
IMP_COMPLETA.zip  425197568
IMP_COMPLETA.zip  425230336
IMP_COMPLETA.zip  425263104
IMP_COMPLETA.zip  425295872
IMP_COMPLETA.zip  425328640
IMP_COMPLETA.zip  425361408
IMP_COMPLETA.zip  425394176
IMP_COMPLETA.zip  425426944
IMP_COMPLETA.zip  425459712
IMP_COMPLETA.zip  42

IMP_COMPLETA.zip  433946624
IMP_COMPLETA.zip  433979392
IMP_COMPLETA.zip  434012160
IMP_COMPLETA.zip  434044928
IMP_COMPLETA.zip  434077696
IMP_COMPLETA.zip  434110464
IMP_COMPLETA.zip  434143232
IMP_COMPLETA.zip  434176000
IMP_COMPLETA.zip  434208768
IMP_COMPLETA.zip  434241536
IMP_COMPLETA.zip  434274304
IMP_COMPLETA.zip  434307072
IMP_COMPLETA.zip  434339840
IMP_COMPLETA.zip  434372608
IMP_COMPLETA.zip  434405376
IMP_COMPLETA.zip  434438144
IMP_COMPLETA.zip  434470912
IMP_COMPLETA.zip  434503680
IMP_COMPLETA.zip  434536448
IMP_COMPLETA.zip  434569216
IMP_COMPLETA.zip  434601984
IMP_COMPLETA.zip  434634752
IMP_COMPLETA.zip  434667520
IMP_COMPLETA.zip  434700288
IMP_COMPLETA.zip  434733056
IMP_COMPLETA.zip  434765824
IMP_COMPLETA.zip  434798592
IMP_COMPLETA.zip  434831360
IMP_COMPLETA.zip  434864128
IMP_COMPLETA.zip  434896896
IMP_COMPLETA.zip  434929664
IMP_COMPLETA.zip  434962432
IMP_COMPLETA.zip  434995200
IMP_COMPLETA.zip  435027968
IMP_COMPLETA.zip  435060736
IMP_COMPLETA.zip  43

IMP_COMPLETA.zip  443645952
IMP_COMPLETA.zip  443678720
IMP_COMPLETA.zip  443711488
IMP_COMPLETA.zip  443744256
IMP_COMPLETA.zip  443777024
IMP_COMPLETA.zip  443809792
IMP_COMPLETA.zip  443842560
IMP_COMPLETA.zip  443875328
IMP_COMPLETA.zip  443908096
IMP_COMPLETA.zip  443940864
IMP_COMPLETA.zip  443973632
IMP_COMPLETA.zip  444006400
IMP_COMPLETA.zip  444039168
IMP_COMPLETA.zip  444071936
IMP_COMPLETA.zip  444104704
IMP_COMPLETA.zip  444137472
IMP_COMPLETA.zip  444170240
IMP_COMPLETA.zip  444203008
IMP_COMPLETA.zip  444235776
IMP_COMPLETA.zip  444268544
IMP_COMPLETA.zip  444301312
IMP_COMPLETA.zip  444334080
IMP_COMPLETA.zip  444366848
IMP_COMPLETA.zip  444399616
IMP_COMPLETA.zip  444432384
IMP_COMPLETA.zip  444465152
IMP_COMPLETA.zip  444497920
IMP_COMPLETA.zip  444530688
IMP_COMPLETA.zip  444563456
IMP_COMPLETA.zip  444596224
IMP_COMPLETA.zip  444628992
IMP_COMPLETA.zip  444661760
IMP_COMPLETA.zip  444694528
IMP_COMPLETA.zip  444727296
IMP_COMPLETA.zip  444760064
IMP_COMPLETA.zip  44

IMP_COMPLETA.zip  453246976
IMP_COMPLETA.zip  453279744
IMP_COMPLETA.zip  453312512
IMP_COMPLETA.zip  453345280
IMP_COMPLETA.zip  453378048
IMP_COMPLETA.zip  453410816
IMP_COMPLETA.zip  453443584
IMP_COMPLETA.zip  453476352
IMP_COMPLETA.zip  453509120
IMP_COMPLETA.zip  453541888
IMP_COMPLETA.zip  453574656
IMP_COMPLETA.zip  453607424
IMP_COMPLETA.zip  453640192
IMP_COMPLETA.zip  453672960
IMP_COMPLETA.zip  453705728
IMP_COMPLETA.zip  453738496
IMP_COMPLETA.zip  453771264
IMP_COMPLETA.zip  453804032
IMP_COMPLETA.zip  453836800
IMP_COMPLETA.zip  453869568
IMP_COMPLETA.zip  453902336
IMP_COMPLETA.zip  453935104
IMP_COMPLETA.zip  453967872
IMP_COMPLETA.zip  454000640
IMP_COMPLETA.zip  454033408
IMP_COMPLETA.zip  454066176
IMP_COMPLETA.zip  454098944
IMP_COMPLETA.zip  454131712
IMP_COMPLETA.zip  454164480
IMP_COMPLETA.zip  454197248
IMP_COMPLETA.zip  454230016
IMP_COMPLETA.zip  454262784
IMP_COMPLETA.zip  454295552
IMP_COMPLETA.zip  454328320
IMP_COMPLETA.zip  454361088
IMP_COMPLETA.zip  45

In [15]:
#check MD5
for file in os.listdir('data/'):
    if file.startswith('EXP_') or file.startswith('IMP_'):
        file_hash = hashlib.md5()
        with open('data/'+file,'rb') as f:
            for chunk in iter(lambda: f.read(2**20*file_hash.block_size), b''):
                file_hash.update(chunk)
        if base64.b64encode(file_hash.digest()) != EXP_MD5 and base64.b64encode(file_hash.digest()) != IMP_MD5:
            print(EXP_MD5)
            print(IMP_MD5)
            print(base64.b64encode(file_hash.digest()))
            raise Exception('Wrong file downloaded!')

In [18]:
#unzip files
for file in os.listdir('data/'):
    if file[-3] == 'zip':
        with zipfile.ZipFile('data/'+file,"r") as zip_ref:
            zip_ref.extractall("data/")
    

We start with exports. One important point we notice are the file sizes. One has around 1.5G unzipped, the other has 2.15G. Those may be a problem for python if you are using a computer with less than 8G RAM Memory.

Let´s import the first one.

In [21]:
exp_df = pd.read_csv('data/EXP_COMPLETA.csv', delimiter=';')

In [22]:
exp_df.head(10)

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
0,1997,3,41043911,15,149,RS,1,1010500,3987,4150,16725
1,1997,5,63019000,10,97,MG,7,145200,0,1002,8420
2,1997,6,87168000,11,586,RS,7,145300,48,153,915
3,1997,8,39162000,10,249,SP,1,817800,0,138,586
4,1997,2,73130000,10,63,SP,7,1010900,0,2714,1972
5,1997,3,20098000,10,756,BA,1,517600,0,12600,5150
6,1997,5,83119000,10,63,MN,4,817600,0,240,6945
7,1997,5,76121000,10,63,RJ,7,1010900,0,4214,25610
8,1997,2,85392990,11,69,SC,1,925100,40,0,72
9,1997,4,39123910,10,628,BA,1,817800,0,10400,70860


We will replace the names with more human friendly ones, and we notice that some of the data is using codes like integers, not in a human friendly naming system. Those will have to be addressed as well. Reading the documentation in http://www.mdic.gov.br/index.php/comercio-exterior/estatisticas-de-comercio-exterior/base-de-dados-do-comercio-exterior-brasileiro-arquivos-para-download , we can download the missing tables and change our data to a more friendly one.

First thing I learned was what FOB means! https://www.ipea.gov.br/desafios/index.php?option=com_content&view=article&id=2115:catid=28&Itemid=23

So, the column VL_FOB should be renamed to Amount(USD). 


